<a href="https://colab.research.google.com/github/TalissaMoura/AmazonC4DatasetAugmented/blob/main/notebooks/1_pipeline_build_hard_queries_with_one_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline for build the hard queries

# 2. Functions

In [1]:
!pip install outlines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00


In [2]:
import os, json, re, torch, pathlib, time
import outlines
from huggingface_hub import login,snapshot_download, upload_file, HfApi
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import userdata
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
from pydantic import BaseModel,ValidationError
from transformers import AutoModelForCausalLM, AutoTokenizer


In [13]:
class ProductReviewOutline(BaseModel):
    positive_points: list[str|None]
    negative_points: list[str|None]

class HardNegativeQuery(BaseModel):
    query: str

def extract_json_from_markdown(response_text: str) -> str:
    """Extract JSON from markdown code blocks"""
    # Look for ```json ... ``` or ``` ... ```
    pattern = r'```(?:json)?\s*(.*?)\s*```'
    matches = re.findall(pattern, response_text, re.DOTALL)

    if matches:
        # Return the first JSON block found
        return matches[0].strip()
    else:
        # If no markdown blocks, return original text
        return response_text

def clean_and_parse_json(response_text: str,pydantic_model:BaseModel) -> BaseModel | None:
    # Step 1: Extract JSON from markdown if present
    json_content = extract_json_from_markdown(response_text)

    # Step 2: Remove any remaining backticks or markdown artifacts
    json_content = json_content.strip()

    # Step 3: Handle potential escaping issues
    if json_content.startswith('"') and json_content.endswith('"'):
        # Unescape if it's still a string representation
        try:
            json_content = json.loads(json_content)
        except json.JSONDecodeError:
            pass  # Keep as is if it's not double-escaped

    # Step 4: Parse and validate
    try:
        if isinstance(json_content, str):
            return pydantic_model.model_validate_json(json_content)
        else:
            return pydantic_model.model_validate(json_content)
    except ValidationError as e:
        print(f"Validation error: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return None

def generate_and_parse_with_retry(
    model,
    tokenizer,
    messages,
    pydantic_model,
    max_attempts=3,
    delay=0.5  # optional: to avoid overwhelming GPU or API rate limits
):
    for attempt in range(1, max_attempts + 1):
        # Tokenize and generate
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=2048
        )
        # Trim input part
        generated_ids = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Try parsing
        try:
            parsed = clean_and_parse_json(response, pydantic_model)
            if parsed is not None:
                return parsed  # success!
        except ValidationError:
            pass  # will retry

        # Optional: small delay before next attempt
        if attempt < max_attempts:
            time.sleep(delay)

    # All attempts failed
    return None  # or raise an exception, depending on desired behavior


def load_model(model_name):
  tokenizer = AutoTokenizer.from_pretrained(model_name,fix_mistral_regex=True,device_map="cpu",)
  model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.float16,device_map="auto")

  # 🔍 Validate placement
  if not next(model.parameters()).is_cuda:
      raise RuntimeError("Model failed to load on GPU!")
  print("✅ Model successfully on GPU.")
  return model,tokenizer


def generate_positive_and_negative_points_for_reviews(df_reviews,
                                                      model,
                                                      tokenizer,
                                                      category,
                                                      model_name,
                                                      save_local=True,
                                                      system_template_path="/content/prompts/system_prompt_review_analysis.txt",
                                                      user_template_path="/content/prompts/user_prompt_review_analysis.txt"):
  batches = df_reviews.to_batches()
  for i,batch in enumerate(batches):
    # Create the parquet dataset
    new_rows = []
    for row in batch.to_pylist():
      positive_review = "\n , ".join(row['ori_positive_review'])
      semi_positive_reviews = "\n , ".join([f"{row['ori_semi_positive_review'][i]}" for i in range(len(row['ori_semi_positive_review']))])

      system_template = outlines.Template.from_file(system_template_path)
      user_template = outlines.Template.from_file(user_template_path)

      messages = [
          {"role": "system", "content": system_template()},
          {"role": "user", "content": user_template(very_positive_review=positive_review,somewhat_positive_review=semi_positive_reviews)}
      ]
      response_treated = generate_and_parse_with_retry(
          model=model,
          tokenizer=tokenizer,
          messages=messages,
          pydantic_model=ProductReviewOutline,
      )
      if response_treated:
        new_rows.append(response_treated)

    batch_pylist = batch.to_pylist()
    new_data = []
    for new_row,data in zip(new_rows,batch_pylist):
        positive_points = new_row.__dict__['positive_points']
        negative_points = new_row.__dict__['negative_points']
        data['positive_points'] = positive_points
        data['negative_points'] = negative_points
        new_data.append(data)

    if save_local:
      if not pathlib.Path(f"/content/datasets/procesed/review_analysis/{model_name}").exists():
        pathlib.Path(f"/content/datasets/procesed/review_analysis/{model_name}").mkdir(parents=True)
      batch = pa.Table.from_pylist(new_data)
      pq.write_table(batch, f"/content/datasets/procesed/review_analysis/{model_name}/{category}_review_analysis_part_{i}.parquet")
      print(f"Batch {i} done.")

def generate_hard_negative_queries(df_reviews,
                                   model,
                                   tokenizer,
                                   category,
                                   model_name,
                                   save_local=True,
                                   system_template_path="/content/prompts/system_prompt_generate_hard_negative_queries.txt",
                                   user_template_path="/content/prompts/user_prompt_generate_hard_negative_queries.txt"):
  batches = df_reviews.to_batches()
  for i,batch in enumerate(batches):
    # Create the parquet dataset
    new_rows = []
    for row in batch.to_pylist():
      positive_char_list = ",".join(row['positive_points'])
      negative_char_list = ",".join(row['negative_points'])

      system_template = outlines.Template.from_file(system_template_path)
      user_template = outlines.Template.from_file(user_template_path)

      for _ in range(3):
        messages = [
            {"role": "system", "content": system_template()},
            {"role": "user", "content": user_template(positive_list=positive_char_list,
                                                      negative_list=negative_char_list)}
        ]


        response_treated = generate_and_parse_with_retry(
            model=model,
            tokenizer=tokenizer,
            messages=messages,
            pydantic_model=HardNegativeQuery,
        )
        if response_treated:
          new_row = {k:v for k,v in row.items()}
          hard_query = response_treated.__dict__['query']
          new_row['hard_negative_query'] = hard_query
          new_rows.append(new_row)


    hq_batch = pa.Table.from_pylist(new_rows)
    if save_local:
      if not pathlib.Path(f"/content/datasets/procesed/hard_negative_queries/{model_name}").exists():
        pathlib.Path(f"/content/datasets/procesed/hard_negative_queries/{model_name}").mkdir(parents=True)
      pq.write_table(hq_batch, f"/content/datasets/procesed/hard_negative_queries/{model_name}/{category}_part_{i}.parquet")
      print(f"Batch {i} done.")

def save_file_in_remote(local_filepath,remote_filepath):
  upload_file(
      path_or_fileobj=f"{local_filepath}",
      path_in_repo=f"{remote_filepath}",
      repo_id="Talissa/AmazonC4Augmented",
      repo_type="dataset",
      commit_message=f"Add {remote_filepath}"
  )
  print("File saved locally")

In [32]:
def pipeline_for_generate_hard_queries(idx_category,model,model_name,tokenizer):

  print(f"Load the reviews for category: {idx_category}")
  try:
    snapshot_download(
        repo_id='Talissa/AmazonC4Augmented',
        repo_type="dataset",
        local_dir='/content/datasets',
        allow_patterns=[f'raw/review_and_meta_categories/filtered_4_star_reviews_{idx_category}.parquet']
    )
  except Exception as e:
    print(f"Error download the file: {e}")

  df_amazonc4 = load_dataset('McAuley-Lab/Amazon-C4')['test'].data.table
  df_reviews = pq.read_table(f'/content/datasets/raw/review_and_meta_categories/filtered_4_star_reviews_{idx_category}.parquet')
  df_all_reviews = df_amazonc4.select(['item_id','ori_review']).rename_columns({'ori_review':'ori_positive_review'}).join(
    df_reviews.select(['asin','ori_review']).rename_columns({'asin':'item_id','ori_review':'ori_semi_positive_review'}),
    join_type='inner',
    keys=['item_id']
    )

  df_final = df_all_reviews.group_by("item_id").aggregate([
      ("ori_semi_positive_review", "list"),
      ("ori_positive_review", "distinct")
  ])

  df_final = df_final.rename_columns({
      'ori_semi_positive_review_list':'ori_semi_positive_review',
      'ori_positive_review_distinct':'ori_positive_review'
  })

  print(f"--- Generate positive and negative points for reviews: {idx_category} ---")
  generate_positive_and_negative_points_for_reviews(df_final,model,tokenizer,idx_category,model_name=model_name)

  df_all_reviews_cat = [
    pq.read_table(f) for f in list(pathlib.Path(f'/content/datasets/procesed/review_analysis/{model_name}').glob(f'{idx_category}_review_analysis_part_*.parquet'))
    ]
  df_batch = pa.concat_tables(df_all_reviews_cat)

  print(f"--- Generate hard negative queries: {idx_category} ---")
  generate_hard_negative_queries(df_batch,model,tokenizer,idx_category,model_name=model_name)


  return f'finish pipeline for category: {idx_category}'


In [8]:
# setup hugging faces
from huggingface_hub import login,snapshot_download, upload_file, HfApi
from google.colab import userdata
import os
HF_TOKEN = userdata.get("HF_TOKEN")
login(HF_TOKEN)
hf_api = HfApi(token=HF_TOKEN)

# 3. Load data

In [5]:
from google.colab import userdata
from huggingface_hub import snapshot_download
from datasets import load_dataset
import pandas as pd
idx_category = 'All_Beauty'
hf_token = userdata.get('HF_TOKEN')
df_amazonc4 = load_dataset('McAuley-Lab/Amazon-C4')['test'].data.table
snapshot_download(
    repo_id='Talissa/AmazonC4Augmented',
    repo_type='dataset',
    local_dir='datasets',
    allow_patterns=[f'raw/review_and_meta_categories/filtered_4_star_reviews_{idx_category}.parquet'],
    token=hf_token,
)

README.md: 0.00B [00:00, ?B/s]

test.csv:   0%|          | 0.00/12.5M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/21223 [00:00<?, ? examples/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

raw/review_and_meta_categories/filtered_(…):   0%|          | 0.00/21.4k [00:00<?, ?B/s]

'/content/datasets'

In [6]:
# Download asin2category file
try:
  snapshot_download(
      repo_id='Talissa/AmazonC4Augmented',
      repo_type='dataset',
      local_dir='datasets',
      allow_patterns=['asin2categoryfiltered.json'],
      token=hf_token,)
except Exception as e:
  print(f'Error download the file: {e}')


with open(f'/content/datasets/asin2categoryfiltered.json', 'r') as f:
  asin2category = json.load(f)

all_categories = list(asin2category.values())
all_categories = list(set(all_categories))
cat_sort = sorted(all_categories)
cat_treated = []
for cat in cat_sort:
  if cat.find(" "):
    cat = cat.replace(" ","_")
    cat_treated.append(cat)
  else:
    cat_treated.append(cat)
print(f"Load all the {len(cat_treated)} categories")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

asin2categoryfiltered.json: 0.00B [00:00, ?B/s]

Load all the 31 categories


# 4. Build the pipeline

1 - Build prompts

In [9]:
import outlines, pathlib

if not pathlib.Path('/content/prompts/').exists():
  pathlib.Path.mkdir('/content/prompts')

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/system_prompt_analysis_review"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    system_template = \
    """
    ```json
      {
      "role": "You are a Review Contrast Analyst specialized in e-commerce feedback.",
      "task": "Compare two sets of user reviews for the same product: one labeled 'very_positive' (high satisfaction), the other 'somewhat_positive' (moderate satisfaction). Extract and contrast the key positive and negative aspects that explain this difference in sentiment.",
      "instructions": [
        "1. Phrase each point as a concise, canonical noun-phrase (e.g., 'strong odor on first use') — avoid verbs like 'is', 'has', 'feels'.",
        "2. For negative points, prefer *constructive phrasing* (e.g., 'initial chemical smell' instead of 'smells bad').",
        "3. Ground every point with an evidence span (short verbatim quote).",
        "4. Do NOT include generic praise (e.g., 'good product', 'recommended') unless tied to a specific feature."
      ],
      "input_format": {
        "very_positive_review": ["string", "..."],
        "somewhat_positive_review": ["string", "..."]
      },
      "output_format": {
        "positive_points": [
            "string — canonical phrase, e.g., 'lightweight and flexible sole'"
        ],
        "negative_points": [
          "string — constructive phrasing, e.g., 'noticeable warmth buildup after 15 minutes'"
        ]
      },
      "requirements": [
        "Max 6 positive and 6 negative points total.",
        "If a point is only implied (not stated), omit it.",
        "If in the review, use where the person use the product (i.e instead of "love these boots" write "love these boots for raining days") ",
        "Prioritize points that *explain the sentiment gap* (e.g., negatives only in 'somewhat_positive' are high-value).",
        "Return ONLY valid JSON. No markdown, no explanations."
      ]
    }
    ```
    """


    with open('/content/prompts/system_prompt_review_analysis.txt', 'w+') as f:
          f.write(system_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/system_prompt_review_analysis.txt",
              path_in_repo=f"prompts/system_template_review_analysis.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add system prompt for review analysis"
          )


No files have been modified since last commit. Skipping to prevent empty commit.


In [10]:
try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/user_prompt_review_analysis"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    user_template = \
    """
    ```json
    {

      "very_positive_review": [{{very_positive_review}}],

      "somewhat_positive_review": [{{somewhat_positive_review}}]

    }
    ```
    """

    with open('/content/prompts/user_prompt_review_analysis.txt', 'w') as f:
          f.write(user_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/user_prompt_review_analysis.txt",
              path_in_repo=f"prompts/user_prompt_review_analysis.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add user prompt for review analysis"
          )


No files have been modified since last commit. Skipping to prevent empty commit.


In [11]:
if not pathlib.Path('/content/prompts/').exists():
  pathlib.Path.mkdir('/content/prompts')

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/system_prompt_generate_hard_negative_queries"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    system_template = \
    """
    ```json
    {
  "task": "You are a text generator. You receive two lists: one containing positive aspects and another containing negative aspects of a product purchased online, from the user's point of view. Based on these lists, generate a first-person, natural-language search query — phrased as if a real person were typing it into a search engine or speaking aloud to a friend.",
  "input_format": {
    "positive_characteristics_list": "[string] — comma-separated list of positive aspects: features, benefits, or qualities the user appreciated.",
    "negative_characteristics_list": "[string] — comma-separated list of negative aspects: drawbacks, failures, or dealbreakers the user experienced."
  },
  "output_type": "str",
  "output_format": {
    "query": "string — a single, fluent, first-person search query in natural spoken/written language"
  },
  "requirements": [
    "The query must sound authentically human: use contractions (e.g., 'I’m', 'don’t'), colloquial phrasing, and optional interjections (e.g., 'Honestly…', 'Looking for…') — but avoid slang or region-specific idioms unless implied by context.",
    "Explicitly embed ONE positive aspect from `positive_list` as a *desired feature* — phrase it functionally (e.g., 'good grip' instead of 'I liked the grip').",
    "Explicitly address ONE negative aspect from `negative_list` as an *avoidance or improvement goal* — rephrase negatives constructively (e.g., 'that doesn’t slip when wet' instead of 'not slippery').",
    "Rephrase the selected positive and negative aspects by adding realistic, everyday context — such as *where*, *when*, or *how* the user plans to use the product — to ground the query in a concrete scenario (e.g., 'I need hiking shoes with solid ankle support that don’t give me blisters after 5 miles on rocky trails' instead of just 'shoes with support and no blisters'). Avoid hypothetical, exaggerated, or unsafe scenarios.",
    "Do NOT list multiple positives/negatives — focus on ONE positive + ONE negative to keep the query focused and high-signal.",
    "Avoid generic terms like 'good', 'great', 'quality' unless directly tied to a concrete attribute (e.g., 'good cushioning' is OK; 'good quality' is not).",
    "The final query must be usable as-is in a real e-commerce search bar or voice assistant — no markdown, quotes, or extra punctuation beyond natural usage.",
    "Return ONLY a valid JSON object with the key `query`. No explanations, prefixes, or suffixes."
  ]
}
```
    """


    with open('/content/prompts/system_prompt_generate_hard_negative_queries.txt', 'w+') as f:
          f.write(system_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/system_prompt_generate_hard_negative_queries.txt",
              path_in_repo=f"prompts/system_prompt_generate_hard_negative_queries.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add system prompt for generate hard negative queries"
          )


No files have been modified since last commit. Skipping to prevent empty commit.


In [12]:
try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/user_prompt_generate_hard_negative_queries"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    user_template = \
    """
    ```json
    positive_characteristics_list -: \n [{{positive_list}}] \n

    negative_characteristics_list -: \n [{{negative_list}}] \n
    ```
    """

    with open('/content/prompts/user_prompt_generate_hard_negative_queries.txt', 'w') as f:
          f.write(user_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/user_prompt_generate_hard_negative_queries.txt",
              path_in_repo=f"prompts/user_prompt_generate_hard_negative_queries.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add user prompt for generate hard negative queries"
          )


No files have been modified since last commit. Skipping to prevent empty commit.


3 - Running pipeline

In [ ]:
if "model" not in globals():
  MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
  model,tokenizer = load_model(MODEL_NAME)
else:
  print("Model already loaded")

selected_cats = [cat_treated[i] for i in range(len(cat_treated))]
print(selected_cats[:1])

for cat in selected_cats[:2]:
  res = pipeline_for_generate_hard_queries(
        idx_category=cat,
        model=model,
        model_name=MODEL_NAME,
        tokenizer=tokenizer
    )
  print(res)

  print("-- Save files in remote -- ")

  ## Save files in remote
  model_name = MODEL_NAME
  base = pathlib.Path(f'/content/datasets/procesed/hard_negative_queries/{model_name}')
  hq_files = [str(f) for f in list(base.glob(f'{cat}_part_*.parquet'))]
  hq_remote_files = [
      f"/datasets/procesed/hard_negative_queries/{model_name}/{cat}_part_{i}.parquet" for i in range(len(hq_files))
  ]

  for local,remote in zip(hq_files,hq_remote_files):
    save_file_in_remote(local,remote)

  print(f"-- save hard negative queries for {cat} --")

  base_review_analysis = pathlib.Path(f'/content/datasets/procesed/review_analysis/{model_name}')
  base_review_analysis_remote = pathlib.Path(f'/datasets/procesed/review_analysis/{model_name}')
  review_analysis_local_files = [str(f) for f in base_review_analysis.glob(f'{cat}_review_analysis_part_*.parquet')]
  review_analysis_remote_files = [f"/datasets/procesed/review_analysis/{model_name}/{cat}_review_analysis_part_{i}.parquet" for i in range(len(review_analysis_local_files))]


  for local,remote in zip(review_analysis_local_files,review_analysis_remote_files):
    print(local,remote)
    save_file_in_remote(local,remote)

  print(f"-- save review analysis for {cat} --")

  print(f"Done for {cat}")

  print(f"Delete files for /datasets/procesed/hard_negative_queries/{model_name}")
  for f in hq_files:
    pathlib.Path(f).unlink(missing_ok=True)
  print(f"Delete files for /datasets/procesed/review_analysis/{model_name}")
  for f in review_analysis_local_files:
    pathlib.Path(f).unlink(missing_ok=True)

  print("-- Clear raw ---")
  base_raw = pathlib.Path('/content/datasets/raw')
  base_raw_local_files = [str(f) for f in base_raw.glob(f'review_and_meta_categories/filtered_4_star_reviews_{cat}*.parquet')]
  for f in base_raw_local_files:
    pathlib.Path(f).unlink(missing_ok=True)

Model already loaded
['All_Beauty']
Load the reviews for category: All_Beauty


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

--- Generate positive and negative points for reviews: All_Beauty ---
Batch 0 done.
--- Generate hard negative queries: All_Beauty ---
Validation error: 1 validation error for HardNegativeQuery
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value="Looking for a hair dryer...hout breaking the bank.", input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid
Validation error: 1 validation error for HardNegativeQuery
  Invalid JSON: expected ident at line 1 column 2 [type=json_invalid, input_value='I’m looking for a colo...niqueness of the scent.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid


In [ ]:
# # Delete models and tokenizer
# import gc
# import torch

# # Delete global variables
# if 'model' in globals():
#     del globals()['model']
#     print("✅ Removed 'model' from globals")

# if 'tokenizer' in globals():
#     del globals()['tokenizer']
#     print("✅ Removed 'tokenizer' from globals")

# if 'model_wrapper' in globals():
#     del globals()['model_wrapper']
#     print("✅ Removed 'model_wrapper' from globals")

# if 'generator' in globals():
#     del globals()['generator']
#     print("✅ Removed 'generator' from globals")

# # Force garbage collection to free CPU memory
# gc.collect()

# # Clear GPU cache to free up CUDA memory
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     print("✅ GPU cache cleared")
# else:
#     print("CUDA not available")

# print("✅ Cleanup complete - model/tokenizer removed from CPU/GPU")

✅ GPU cache cleared
✅ Cleanup complete - model/tokenizer removed from CPU/GPU
